## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf

### Collect Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
pricing = pd.read_csv('/content/drive/My Drive/AIML_NoteBooks/LabAndAssignments/R6/prices.csv')

### Check all columns in the dataset

In [5]:
pricing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
pricing = pricing.drop(['date','symbol'],axis=1)

### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
pricing['volume'] = pricing['volume'].apply(lambda x: x/1000000)

In [0]:
pricing = pricing.head(1000)

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

X =  pricing.drop("volume", axis=1)
y =  pricing.pop("volume")
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.30, random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn import preprocessing
X_normalized = preprocessing.normalize(X, norm='l2')

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')
x_n = tf.nn.l2_normalize(x,1) 
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

2.Define a function to calculate prediction

In [0]:
y = tf.add(tf.matmul(x_n,W),b,name='output') #X is normalized, hence X_n

3.Loss (Cost) Function [Mean square error]

In [0]:
loss_func = tf.reduce_mean(tf.square(y-y_),name='Loss')

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss_func)

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
training_epochs = 100

### Get the shapes and values of W and b

In [17]:
print("Shape of W : ",W.shape)
print("Shape of b : ",b.shape)

Shape of W :  (4, 1)
Shape of b :  (1,)


### Model Prediction on 1st Examples in Test Dataset

In [18]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss_func],feed_dict={x:X_train, y_:y_train}) # X and Y is like X_Train and Y_Train here.
    
    if epoch % 5 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  225.87958
Training loss at step:  5  is  203.89143
Training loss at step:  10  is  197.76749
Training loss at step:  15  is  196.0619
Training loss at step:  20  is  195.58687
Training loss at step:  25  is  195.45459
Training loss at step:  30  is  195.41774
Training loss at step:  35  is  195.4075
Training loss at step:  40  is  195.40462
Training loss at step:  45  is  195.40382
Training loss at step:  50  is  195.4036
Training loss at step:  55  is  195.40355
Training loss at step:  60  is  195.40353
Training loss at step:  65  is  195.40355
Training loss at step:  70  is  195.40353
Training loss at step:  75  is  195.40353
Training loss at step:  80  is  195.40353
Training loss at step:  85  is  195.40353
Training loss at step:  90  is  195.40355
Training loss at step:  95  is  195.40353


In [19]:
test_loss = sess.run([loss_func],feed_dict={x:X_test, y_:y_test}) 
print ('Training loss at step:is ', test_loss)

Training loss at step:is  [241.01949]


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [20]:
data = pd.read_csv('/content/drive/My Drive/AIML_NoteBooks/LabAndAssignments/R6/11_Iris.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [21]:
data_new = pd.get_dummies(data,prefix=['Species'],columns=['Species'])
data_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
Id                         150 non-null int64
SepalLengthCm              150 non-null float64
SepalWidthCm               150 non-null float64
PetalLengthCm              150 non-null float64
PetalWidthCm               150 non-null float64
Species_Iris-setosa        150 non-null uint8
Species_Iris-versicolor    150 non-null uint8
Species_Iris-virginica     150 non-null uint8
dtypes: float64(4), int64(1), uint8(3)
memory usage: 6.4 KB


### Splitting the data into feature set and target set

In [0]:
from sklearn.model_selection import train_test_split

X =  data_new.drop(['Id','Species_Iris-setosa','Species_Iris-versicolor','Species_Iris-virginica'], axis=1)
y =  data_new[['Species_Iris-setosa','Species_Iris-versicolor','Species_Iris-virginica']]
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.30, random_state=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [23]:
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


### Model Training 

In [24]:
    model = Sequential()
    model.add(Dense(3, input_dim=4, kernel_initializer='normal', activation='softmax'))#by default, kernel_initializer will be uniform distribution.
    #model.add(Dense(3, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics = ['accuracy']) #try this with adam or sgv

W0721 11:39:28.544868 139736798652288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0721 11:39:28.548286 139736798652288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0721 11:39:28.552772 139736798652288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0721 11:39:28.569170 139736798652288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0721 11:39:28.587576 139736798652288 deprecation_wrapper.

### Model Prediction

In [25]:
model.fit(X_train, y_train,validation_data = (X_test, y_test), epochs=10, batch_size=5)
scores = model.evaluate(X,y)

W0721 11:39:28.689574 139736798652288 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0721 11:39:28.737172 139736798652288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 105 samples, validate on 45 samples
Epoch 1/10
105/105 [==============================] - 0s 2ms/step - loss: 1.0376 - acc: 0.4857 - val_loss: 1.0026 - val_acc: 0.6000
Epoch 2/10
105/105 [==============================] - 0s 625us/step - loss: 0.9186 - acc: 0.6952 - val_loss: 0.8815 - val_acc: 0.7111
Epoch 3/10
105/105 [==============================] - 0s 522us/step - loss: 0.8605 - acc: 0.6952 - val_loss: 0.8406 - val_acc: 0.6000
Epoch 4/10
105/105 [==============================] - 0s 559us/step - loss: 0.7706 - acc: 0.7238 - val_loss: 0.7612 - val_acc: 0.6000
Epoch 5/10
105/105 [==============================] - 0s 501us/step - loss: 0.7381 - acc: 0.7619 - val_loss: 0.7604 - val_acc: 0.6000
Epoch 6/10
105/105 [==============================] - 0s 523us/step - loss: 0.6869 - acc: 0.7333 - val_loss: 0.6754 - val_acc: 0.7556
Epoch 7/10
105/105 [==============================] - 0s 585us/step - loss: 0.6448 - acc: 0.7905 - val_loss: 0.7404 - val_acc: 0.6000
Epoch 8/10
105/105 

### Save the Model

In [26]:
scores[1]*100

67.33333333333333

In [0]:
model.save('output.dump')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?